In [ ]:
import re, time, requests
from fastcore.all import *
from urllib.parse import urljoin

def search_images(term, max_images=5):
    print(f"🔍 Searching for: {term}")
    
    url = 'https://duckduckgo.com/'
    res = requests.post(url, data={'q': term}, timeout=5)
    search_obj = re.search(r'vqd=([\d-]+)&', res.text)
    if not search_obj:
        raise Exception('❌ Failed to get search token (vqd). Try again later or check your internet.')
    
    token = search_obj.group(1)
    params = dict(l='us-en', o='json', q=term, vqd=token, f='', p='1', v7exp='a')
    request_url = url + 'i.js'
    
    urls, seen = L(), set()
    first_request = True
    
    while len(urls) < max_images:
        try:
            if first_request:
                res = requests.get(
                    request_url,
                    params=params,
                    headers={
                        'referer': 'https://duckduckgo.com',
                        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0 Safari/537.36'
                    },
                    timeout=10
                )
                first_request = False
            else:
                res = requests.get(
                    request_url,
                    headers={
                        'referer': 'https://duckduckgo.com',
                        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0 Safari/537.36'
                    },
                    timeout=10
                )
            
            res.raise_for_status()
            data = res.json()
            
            new_urls = [item['image'] for item in data.get('results', []) if item['image'] not in seen]
            urls += new_urls
            seen.update(new_urls)
            
            print(f"Fetched {len(urls)} images so far...")
            
            if 'next' not in data or not data['next']:
                break
            
            request_url = urljoin(url, data['next'])
            print(f"Next URL: {request_url}")
            
            time.sleep(0.2)
        
        except Exception as e:
            print(f"⚠️ Error fetching images: {e}")
            break
    
    print(f"✅ Found {len(urls)} image(s).")
    return urls[:max_images]


In [13]:
urls = search_images('bird photos', max_images=5)
print(urls[0])


🔍 Searching for: bird photos


ConnectTimeout: HTTPSConnectionPool(host='duckduckgo.com', port=443): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x1087e0e10>, 'Connection to duckduckgo.com timed out. (connect timeout=None)'))

In [8]:
from fastdownload import download_url
from PIL import Image

dest = 'bird.jpg'
download_url(urls[0], dest, show_progress=True)

im = Image.open(dest)
im.show()  # Opens image in default image viewer, or use im.to_thumb(256,256) in notebooks


NameError: name 'urls' is not defined